#Choco Tutorial - Modifying the core for an application


Here we will learn how to make a simple application using Choco. This requires us first studying the structure of applications.

###Understanding the Application structure 
First, let's go through the structure of an application going through the code of test_choco.c

First we define the global variables and structures we will need in each node for our code to work. In this case, we want to keep record of the sampling sequence, and define the structure of the sensor data packet we will send to the sink node.

Then, we define the functions that will manage different outputs. In this case we only have 1 type of packet, so we will just call a handler that will print the data of that packet. In case we have different packet types that could be sent, each one should have a handler.

Here we can define the processes that will run during our choco execution. We can later pick which nodes will run each process. We can also call PROCESS_YIELD_UNTIL_NEXT_SLOT() and PROCESS_YIELD_SLOTS(int i) to have a better use of the time slots.

This is the on_recv method of choco. In choco-sched.h we can see that there are several functions that can be extended, which we can find in choco-sched.h: 

extern void choco_on_booted(void);
extern void choco_on_synced(void);
extern void choco_on_id_assigned(void);
extern void choco_on_nodes_changed(node_id_t nodes, node_id_t changed_node);
extern void choco_on_recv(node_id_t node_id, uint8_t type, void* d, uint8_t len);

These functions allow the application to take control and run relevant code in critical moments of the execution.

This choco_on_synced() example show us how to get the sampling_seq and start a new process at a defined time. In this case every node synchronized will start the sensing_process, but we can query for IDs or IS_ROOT() to select only some nodes to run a certain process.

### Getting data from the Choco core

An important part of developing a new application is getting data from the process themselves. For that, we're going to create new functions to collect data from the choco process. In particular, we want each node to report the RSSI detected from their neighbors.  

For this, we first need to find where the data is. Usually, the files choco-sched.c, choco-sched.h, glossy-driver.c and glossy-driver.h, located in choco/core/dev/, will have all relevant implementation we will want to go through.

We can see our relevant code about rssi and neighbors is in choco-sched.c. What we need to do then is

 1) Find the relevant variables: where they are assigned and when they are restarted
 
 2) Create variables which keep the values of our relevant ones without restarting
 
 3) Make them accessible from the application layer through some function.
 
 
In our case we can start by creating a structure that will help us keep the data for all nodes. Since we want an ordered output and we're not extremely worried about space, we will create a structure with the origin node id, and an array of length CHOCO_MAX_NODES to save the RSSI values, using the node_id as the key. 

To make this accessible, we will add this in the choco-sched.h file, which is imported by all applications.

Also, we'll define a function which will provide an interface to write the data to our structure, and to get it from the application side:

Now we will move to choco-sched.c to implement this function and assign the values. We need to instantiate our structure as a variable in the node, write both functions, and find where to run the fix_neighbors_rssi() one to write the data in the right time. This way we can start by defining our variable:

The implementation of our function, then, will be just to return this value. We now need to find where in the code to add the assignment of values for our structure. We will see this information is processed in the nw_stat_process, which we should modify as follows:

Now we're done! We can now write our application, taking test_choco.c as a base.

###Using our modifications

Using the structure previously mentioned for applications, we will now retrieve the data from the choco core, send it to the sink node and let it print the packets.
To start we will just print the nodes and the RSSI reported between them. Extending our structure and algorithm to also print the battery level is proposed!

Now that we have ours structure and actions over packets received, we can write our process where we will call the get_neighbors_rssi() function, parse every result and make a packet for each neighboring pair.

And there we go! We will get reports of the RSSI between nodes :) we made our first data gathering extension and wrote an application to use it!